In [3]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [4]:
import stable_nalu

In [5]:
import math
import torch
import stable_nalu
import argparse
import tqdm

In [6]:
%run stable_nalu/experiments/simple_function_static.py

ERROR:root:File `'stable_nalu/experiments/simple_function_static.py'` not found.


In [8]:
dataset = stable_nalu.dataset.NumberTranslationDataset(
    use_cuda=True
)

In [9]:
dataset_train = dataset.fork(subset='train').dataloader()
dataset_valid = dataset.fork(subset='valid').dataloader()

In [10]:
model = stable_nalu.network.NumberTranslationNetwork(
    'NALU',
    writer=None
)

In [11]:
model.cuda()

NumberTranslationNetwork(
  unit_name=NALU, embedding_size=2, hidden_size=2, dictionary_size=30
  (embedding): Embedding(30, 2)
  (lstm_cell): LSTMCell(2, 2)
  (output_cell): GeneralizedCell(
    input_size=2, hidden_size=1, unit_name=NALU
    (cell): NALUCell(
      input_size=2, hidden_size=1
      (op): NALULayer(
        in_features=3, out_features=1, eps=1e-07, nalu_two_nac=False, nalu_bias=False
        (nac_add): NACLayer(in_features=3, out_features=1)
      )
    )
  )
)

In [12]:
model.reset_parameters()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [13]:
def test_model(dataloader):
    with torch.no_grad(), model.no_internal_logging():
        acc_loss = 0
        for x, t in dataloader:
            # forward
            y = model(x)
            acc_loss += criterion(y, t).item() * len(t)

        return acc_loss / len(dataloader.dataset)

In [23]:
# Train model
global_step = 0
for epoch_i in tqdm.tqdm_notebook(range(10000)):
    for i_train, (x_train, t_train) in enumerate(dataset_train):
        global_step += 1
#         summary_writer.set_iteration(global_step)

        # Prepear model
        model.set_parameter('tau', max(0.5, math.exp(-1e-5 * global_step)))
        optimizer.zero_grad()

#         # Log validation
#         if epoch_i % 50 == 0 and i_train == 0:
#             summary_writer.add_scalar('loss/valid', test_model(dataset_valid))

        # forward
        y_train = model(x_train)
        loss_train_criterion = criterion(y_train, t_train)
        loss_train_regualizer = 0.1 * (1 - math.exp(-1e-5 * global_step)) * model.regualizer()[0]
        loss_train = loss_train_criterion + loss_train_regualizer

        # Log loss
#         summary_writer.add_scalar('loss/train/critation', loss_train_criterion)
#         summary_writer.add_scalar('loss/train/regualizer', loss_train_regualizer)
#         summary_writer.add_scalar('loss/train/total', loss_train)
        if epoch_i % 50 == 0 and i_train == 0:
            print(f'train {epoch_i}: {loss_train_criterion}')

        # Optimize model
        if loss_train.requires_grad:
            loss_train.backward()
            optimizer.step()
        model.optimize(loss_train_criterion)

        # Log gradients if in verbose mode
        if True and epoch_i % 10 == 0 and i_train == 0:
            model.log_gradients()




C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


train 0: 375721.21875


KeyboardInterrupt: 

In [ ]:
# Compute losses
loss_train = test_model(dataset_train)
loss_valid = test_model(dataset_valid)

# Write results for this training
print(f'finished:')
print(f'  - loss_train: {loss_train}')
print(f'  - loss_valid: {loss_valid}')

# save results
results_writer.add({
    'seed': args.seed,
    'layer_type': args.layer_type,
    'cuda': args.cuda,
    'verbose': args.verbose,
    'max_epochs': args.max_epochs,
    'loss_train': loss_train,
    'loss_valid': loss_valid,
})